In [5]:
import glob
import numpy as np
import pandas as pd
import parselmouth

from parselmouth.praat import call
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [6]:
# This is the function to measure voice pitch
def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    

    return meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

In [7]:
def runPCA(df):
    #Z-score the Jitter and Shimmer measurements
    features = ['localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter',
                'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer']
    # Separating out the features
    x = df.loc[:, features].values
    # Separating out the target
    #y = df.loc[:,['target']].values
    # Standardizing the features
    x = StandardScaler().fit_transform(x)
    #PCA
    pca = PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,svd_solver='full', tol=0.0, whiten=False)
    principalComponents = pca.fit_transform(x)
    #principalComponents = sklearn_pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['JitterPCA', 'ShimmerPCA'])
    principalDf
    
    #vals_std = StandardScaler().fit_transform(vals)

    #sklearn_pca = PCA(n_components = 'however many you want')

   # vals_pca = sklearn_pca.fit_transform(vals_std)
    return principalDf

In [8]:
# create lists to put the results
file_list = []
mean_F0_list = []
sd_F0_list = []
hnr_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
ddpJitter_list = []
localShimmer_list = []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list = []
ddaShimmer_list = []

# Go through all the wave files in the folder and measure pitch

for wave_file in glob.glob("*.wav"):
    print(wave_file)
    sound = parselmouth.Sound(wave_file)
    (meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(sound, 75, 500, "Hertz")
    file_list.append(wave_file) # make an ID list
    mean_F0_list.append(meanF0) # make a mean F0 list
    sd_F0_list.append(stdevF0) # make a sd F0 list
    hnr_list.append(hnr)
    localJitter_list.append(localJitter)
    localabsoluteJitter_list.append(localabsoluteJitter)
    rapJitter_list.append(rapJitter)
    ppq5Jitter_list.append(ppq5Jitter)
    ddpJitter_list.append(ddpJitter)
    localShimmer_list.append(localShimmer)
    localdbShimmer_list.append(localdbShimmer)
    apq3Shimmer_list.append(apq3Shimmer)
    aqpq5Shimmer_list.append(aqpq5Shimmer)
    apq11Shimmer_list.append(apq11Shimmer)
    ddaShimmer_list.append(ddaShimmer)
df = pd.DataFrame(np.column_stack([file_list, mean_F0_list, sd_F0_list, hnr_list, localJitter_list, localabsoluteJitter_list, rapJitter_list, ppq5Jitter_list, ddpJitter_list, localShimmer_list, localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, apq11Shimmer_list, ddaShimmer_list]),
                               columns=['voiceID', 'meanF0Hz', 'stdevF0Hz', 'HNR', 'localJitter', 'localabsoluteJitter', 'rapJitter', 
                                        'ppq5Jitter', 'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 
                                        'apq11Shimmer', 'ddaShimmer'])  #add these lists to pandas in the right order
pcaData = runPCA(df)

#df = pd.concat([df, pcaData], axis=1)

# Write out the updated dataframe
#df.to_csv("processed_results.csv", index=False)

audio1.wav
audio2.wav
audio3.wav
final1.wav
final2.wav
final3.wav
sample1_wav.wav
sample2_wav.wav


C:\Users\Namrata\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Namrata\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


sample3_wav.wav


In [9]:
df

,voiceID,meanF0Hz,stdevF0Hz,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer
0,audio1.wav,255.01832144118833,34.23612794300979,10.635812071659668,0.027420150922812307,0.00010895288924579658,0.013779566088019482,0.012678044119424466,0.04133869826405845,0.16480913787147053,1.44861417296732,0.0662228095861884,0.10441869487286204,0.18905637122076357,0.1986684287585652
1,audio2.wav,253.76348422800373,41.04683775433522,11.396517483611929,0.013153293361812073,5.148239183779252e-05,0.005403784159794686,0.0046405166632785995,0.01621135247938406,0.12208917956016686,1.1140829439863578,0.05738089907895521,0.08227996696080313,0.134302351058611,0.17214269723686562
2,audio3.wav,197.7131227759748,51.245167790670266,7.0108465253173335,0.027663641478088793,0.0001399891831055806,0.014106889478868583,0.015334423547503641,0.04232066843660575,0.18735734194411888,1.7369332374139443,0.09058661778218285,0.13688530720302983,0.3188315927248891,0.27175985334654856
3,final1.wav,221.60325186605306,61.14786535057858,9.004377618904922,0.02061528876323424,9.35994183587676e-05,0.009767107458964287,0.01049234555663544,0.029301322376892862,0.12885589154677127,1.279782072563812,0.06598401940612567,0.08676191167924978,0.13191405049055857,0.197952058218377
4,final2.wav,236.7028073244553,44.14680719181198,13.375787174517844,0.023998675610464076,0.0001009148513262248,0.011070722221591181,0.012939055449970068,0.03321216666477354,0.1155189554439113,1.158810237573333,0.047355051390936063,0.06457833960355548,0.12155601538600813,0.14206515417280818
5,final3.wav,229.28273532861542,42.78392946973285,13.162042765865504,0.020413456681909695,8.876545748307843e-05,0.009401711307288868,0.010082795590925378,0.028205133921866604,0.1366439407780633,1.3135211073802493,0.0550002940978296,0.07349279349212624,0.12799713440381727,0.1650008822934888
6,sample1_wav.wav,310.1161252911025,68.96243181258946,13.004998258928454,0.008083761937912146,2.598102578130824e-05,0.0033241090787032197,0.0037762522150954383,0.00997232723610966,0.10858357183249558,0.9757902720879486,0.050189578071588414,0.06670681563692996,0.09190307761215216,0.15056873421476524
7,sample2_wav.wav,335.85990845311613,80.61284107275875,13.433162955442677,0.013214419277260886,3.9687703000667514e-05,0.005863079138177156,0.00589792223060476,0.017589237414531465,0.12571111275862878,1.1385403357450226,0.05556263823555593,0.07924181639739025,0.12198784460702577,0.1666879147066678
8,sample3_wav.wav,319.21408637894024,107.40005008143514,9.74627807077706,0.032323396797505746,0.00010200919600344213,0.017727880178727177,0.019596179704249206,0.053183640536181534,0.14885287994563295,1.335206149116504,0.06073964047284532,0.0853039693787258,0.14876203267850235,0.18221892141853596


In [6]:
df.describe()

,voiceID,meanF0Hz,stdevF0Hz,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer
count,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
unique,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
top,sample2_wav.wav,310.1161252911025,80.61284107275875,9.74627807077706,0.013214419277260886,2.598102578130824e-05,0.017727880178727177,0.00589792223060476,0.017589237414531465,0.14885287994563295,0.9757902720879486,0.06073964047284532,0.07924181639739025,0.09190307761215216,0.15056873421476524
freq,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [7]:
pcaData

,JitterPCA,ShimmerPCA
0,-3.583849,-0.581116
1,-0.702390,0.916771
2,4.286239,-0.335656


In [8]:
df.to_csv("test10.csv")  
#read it back
pd.read_csv("test10.csv").head()

,Unnamed: 0,voiceID,meanF0Hz,stdevF0Hz,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer
0,0,sample1_wav.wav,310.116125,68.962432,13.004998,0.008084,0.000026,0.003324,0.003776,0.009972,0.108584,0.975790,0.050190,0.066707,0.091903,0.150569
1,1,sample2_wav.wav,335.859908,80.612841,13.433163,0.013214,0.000040,0.005863,0.005898,0.017589,0.125711,1.138540,0.055563,0.079242,0.121988,0.166688
2,2,sample3_wav.wav,319.214086,107.400050,9.746278,0.032323,0.000102,0.017728,0.019596,0.053184,0.148853,1.335206,0.060740,0.085304,0.148762,0.182219
